In [1]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
from transformers import  pipeline
from langchain.llms import HuggingFacePipeline
model = "C:/Users/arasu/Workspace/Projects/GenAI/models/MBZUAILaMini-Flan-T5-248M/"

tokenizer = T5Tokenizer.from_pretrained(model,truncation=True)
base_model = T5ForConditionalGeneration.from_pretrained(model)

pipe = pipeline(
        'text2text-generation',
        # 'summarization',
        model = base_model,
        tokenizer = tokenizer,
        max_length = 500,
        do_sample = True,
        temperature = 0.5,
        top_p= 0.95
    )
llm = HuggingFacePipeline(pipeline=pipe)

c:\Users\arasu\Workspace\Projects\GenAI\10.STB-6000_chat\vnev\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [17]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
text = []

# loader = PyPDFLoader("/content/STB-6000_user_manual_texts.pdf")
# text.extend(loader.load())
loader = TextLoader("C:/Users/arasu/Workspace/Projects/GenAI/10.STB-6000_chat/manual/STB-6000_USER_MANUAL.txt")
text.extend(loader.load())
text_spiltter = RecursiveCharacterTextSplitter(chunk_size = 1400, chunk_overlap = 350)
text_documents = text_spiltter.split_documents(text)


In [20]:
text_documents[1]

Document(page_content="The STB-6000 USER MANUAL provides clear instructions for setting up your HD Set Top Box. In the package, you will find the STB-6000 HD Set Top Box, a remote control, an AV Cable (Composite RCA), and, of course, the User Manual (which is the document you're reading). To begin, installing the batteries in the remote control is the first step. Simply remove the battery compartment cover located on the back of the remote control, insert the two AAA batteries provided, ensuring the correct polarity as indicated in the compartment. Once the batteries are in place, put the cover back on and ensure it locks securely. This ensures your remote control is ready for use. The User Manual offers detailed yet straightforward guidance, making the setup process easy for users to follow and understand.", metadata={'source': 'C:/Users/arasu/Workspace/Projects/GenAI/10.STB-6000_chat/manual/STB-6000_USER_MANUAL.txt'})

In [2]:
from langchain.embeddings import HuggingFaceBgeEmbeddings
embedding = HuggingFaceBgeEmbeddings(model_name="C:/Users/arasu/Workspace/Projects/GenAI/embeddings/hkunlp_instructor-large/")


In [3]:
from langchain.vectorstores import Chroma
db_path = "text_vector_db"
text_vector = Chroma(persist_directory=db_path,embedding_function=embedding)

In [4]:
retriever = text_vector.as_retriever(search_kwargs={"k":2})

In [5]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever
bm25_retriever = BM25Retriever.from_documents(text_documents)
ensemble_retriever = EnsembleRetriever(retrievers=[retriever, bm25_retriever], weights=[0.5, 0.5])

NameError: name 'text_documents' is not defined

In [9]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
template = """
    You are friendly customer care assistant trying to help user on the context provided.\
    give the answer in a very detailed manner.\
    if the answer is not found in the context then reply "I Dont Know".\
    context: {context}
    question: {question}
    """
prompt = PromptTemplate(template=template, input_variables=["context", "question"])

chain_type_kwargs = {"prompt": prompt}
qa = RetrievalQA.from_chain_type(
    llm = llm,
    chain_type = "stuff",
    retriever = retriever,
    input_key="query",
    chain_type_kwargs=chain_type_kwargs
)

In [6]:
result = qa("how to install batteries?")
print(result)

c:\Users\arasu\Workspace\Projects\GenAI\10.STB-6000_chat\vnev\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


{'query': 'what are all the ports available in the rear panel of the setup box?', 'result': 'The ports available in the rear panel of the setup box are Antenna In (RF IN), Antenna Loop Through (RF OUT), HDMI Output, Video Output, and two Audio Left/Right outputs.'}
